# 📚 RAG: Documentación Técnica con LLMs

Objetivo: construir un sistema RAG (Retrieval Augmented Generation) para consultar documentación de datos, diccionarios de esquemas, y knowledge bases.

- Duración: 120 min
- Dificultad: Alta
- Stack: OpenAI, ChromaDB, LangChain

## 1. Setup ChromaDB

In [ ]:
# pip install chromadb openai langchain
import chromadb
from chromadb.config import Settings

# Cliente persistente
client = chromadb.PersistentClient(path='./chroma_db')

# Crear colección
collection = client.get_or_create_collection(
    name='data_docs',
    metadata={'description': 'Documentación técnica de datos'}
)

print(f'Colección creada: {collection.name}')
print(f'Documentos: {collection.count()}')

## 2. Ingestión de documentación

In [ ]:
import os
from openai import OpenAI
client_openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def get_embedding(text: str) -> list[float]:
    """Genera embedding con OpenAI."""
    resp = client_openai.embeddings.create(
        model='text-embedding-ada-002',
        input=text
    )
    return resp.data[0].embedding

docs = [
    {
        'id': 'tabla_ventas',
        'text': '''
Tabla: ventas
Esquema: dwh.ventas
Descripción: Transacciones de ventas diarias desde 2020.
Columnas:
- venta_id (BIGINT, PK): identificador único
- fecha (DATE): fecha de la transacción
- producto_id (INT, FK): referencia a dim_productos
- cantidad (INT): unidades vendidas
- total (DECIMAL): monto en USD
Frecuencia: actualización diaria a las 3 AM
Owner: equipo-analytics
        ''',
        'metadata': {'type': 'schema', 'owner': 'analytics'}
    },
    {
        'id': 'pipeline_ventas',
        'text': '''
Pipeline: ventas_daily_etl
Descripción: procesa ventas del día anterior
Pasos:
1. Extracción de S3 (bucket: raw-data/ventas/)
2. Validación con Great Expectations
3. Deduplicación por venta_id
4. Enriquecimiento con datos de productos
5. Carga a Redshift
Dependencias: dim_productos debe estar actualizado
Alertas: email a data-eng si falla
        ''',
        'metadata': {'type': 'pipeline', 'owner': 'data-eng'}
    },
    {
        'id': 'metrica_revenue',
        'text': '''
Métrica: monthly_revenue
Definición: SUM(total) de ventas agrupado por mes
Fórmula: SELECT DATE_TRUNC('month', fecha) mes, SUM(total) revenue FROM ventas GROUP BY 1
Business owner: CFO
Dashboards: Tableau (Revenue Overview)
        ''',
        'metadata': {'type': 'metric', 'owner': 'finance'}
    }
]

# Agregar a ChromaDB
for doc in docs:
    embedding = get_embedding(doc['text'])
    collection.add(
        ids=[doc['id']],
        documents=[doc['text']],
        embeddings=[embedding],
        metadatas=[doc['metadata']]
    )

print(f'✅ {len(docs)} documentos indexados')

## 3. Búsqueda semántica

In [ ]:
def semantic_search(query: str, top_k: int = 3) -> list:
    """Busca documentos relevantes."""
    query_embedding = get_embedding(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results

pregunta = '¿Qué tabla contiene información de ventas?'
resultados = semantic_search(pregunta)

print(f'Pregunta: {pregunta}\n')
for i, doc in enumerate(resultados['documents'][0]):
    print(f'Resultado {i+1}:')
    print(doc[:200] + '...')
    print()

## 4. RAG: respuesta con contexto

In [ ]:
def rag_answer(question: str) -> str:
    """Responde usando RAG."""
    # 1. Buscar contexto relevante
    results = semantic_search(question, top_k=2)
    context = '\n\n'.join(results['documents'][0])
    
    # 2. Prompt con contexto
    prompt = f'''
Eres un experto en ingeniería de datos. Responde la pregunta usando SOLO la información del contexto.

Contexto:
{context}

Pregunta: {question}

Respuesta (menciona la fuente si es relevante):
'''
    
    # 3. Generar respuesta
    resp = client_openai.chat.completions.create(
        model='gpt-4',
        messages=[{'role': 'user', 'content': prompt}],
        temperature=0.1
    )
    
    return resp.choices[0].message.content.strip()

preguntas = [
    '¿Cuál es el esquema de la tabla de ventas?',
    '¿A qué hora se actualiza la data de ventas?',
    '¿Qué pipeline procesa las ventas?',
    '¿Cómo se calcula el monthly revenue?'
]

for q in preguntas:
    answer = rag_answer(q)
    print(f'❓ {q}')
    print(f'✅ {answer}\n')

## 5. RAG con LangChain

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# Embeddings y vectorstore
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'))
vectorstore = Chroma(
    persist_directory='./chroma_db',
    embedding_function=embeddings,
    collection_name='data_docs'
)

# LLM
llm = ChatOpenAI(model='gpt-4', temperature=0, openai_api_key=os.getenv('OPENAI_API_KEY'))

# Chain RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vectorstore.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True
)

result = qa_chain({'query': '¿Quién es el owner de la tabla ventas?'})
print('Respuesta:', result['result'])
print('\nFuentes:')
for doc in result['source_documents']:
    print('-', doc.metadata)

## 6. Chunking avanzado

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Para documentos largos
long_doc = '''
# Data Warehouse - Guía Completa

## Arquitectura
Redshift cluster con 5 nodos dc2.large.
Schemas: raw, staging, dwh, analytics.

## Tablas principales
- ventas: transacciones diarias
- clientes: información demográfica
- productos: catálogo completo

## Pipelines
Airflow con 15 DAGs ejecutándose diariamente.
'''

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    separators=['\n\n', '\n', '. ', ' ']
)

chunks = splitter.split_text(long_doc)
print(f'Documento dividido en {len(chunks)} chunks:\n')
for i, chunk in enumerate(chunks):
    print(f'Chunk {i+1}: {chunk}\n')

## 7. Filtrado por metadatos

In [ ]:
# Buscar solo pipelines
results_pipeline = collection.query(
    query_embeddings=[get_embedding('automatización de datos')],
    n_results=5,
    where={'type': 'pipeline'}
)

print('Pipelines encontrados:')
for doc in results_pipeline['documents'][0]:
    print('-', doc.split('\n')[1])

## 8. Buenas prácticas RAG

- **Chunking inteligente**: divide por secciones lógicas (headers, párrafos).
- **Metadatos ricos**: agrega source, timestamp, owner, versión.
- **Híbrido**: combina búsqueda semántica + keyword search.
- **Re-ranking**: usa modelos como Cohere Rerank para mejorar resultados.
- **Cache**: guarda respuestas frecuentes.
- **Actualización**: sincroniza vectorstore con cambios en docs.
- **Monitoreo**: loggea queries, latencia, quality de respuestas.

## 9. Ejercicios

1. Indexa tu documentación real de data warehouse en ChromaDB.
2. Construye un chatbot Slack que responda preguntas sobre esquemas.
3. Implementa hybrid search (semántico + BM25) con LangChain.
4. Crea un dashboard Streamlit para explorar el vectorstore.